# Torben data

In [ ]:
from src.data.utils import read_ts, get_random_ts
from src.visualization.time_series import plot_ts

In [ ]:
ts = get_random_ts()
ts

In [ ]:
ts_neg = ts.sel(date=slice('2020-02-24', '2022-02-24'))
ts_pos = ts.sel(date=slice('2021-02-24', '2023-02-24'))
print(ts_pos.shape, ts_neg.shape)

In [ ]:
ts.sel(date=slice('2020-02-24', '2022-02-24')).shape

In [ ]:
from src.data import load_unosat_labels
df = load_unosat_labels('UKR1')
df

In [ ]:
import h5py

In [ ]:
ts.attrs

In [ ]:
ts.aoi

In [ ]:
for attr in ts.attrs:
    print(attr, type(ts.attrs[attr]))

In [ ]:
ts = ts.sel(date=slice('2022-01-24', '2022-04-24'))
ts.date

In [ ]:
import numpy as np
import datetime as dt

target_date = np.datetime64(UKRAINE_WAR_START)
np.max(np.where(ts.date < target_date)), np.min(np.where(ts.date > np.datetime64(ts.date_of_analysis)))

In [ ]:
from src.constants import AOIS_TEST, UKRAINE_WAR_START
UKRAINE_WAR_START

In [ ]:
list(folder.glob('*.nc'))

In [ ]:
ts.shape

In [ ]:
import h5py
from src.constants import AOIS_TEST, UKRAINE_WAR_START
from tqdm import tqdm
from src.data.utils import aoi_orbit_iterator
from src.constants import TS_PATH
import xarray as xr

extraction_strategy = '3x3'

fps = []
for aoi, orbit in aoi_orbit_iterator():
    folder = TS_PATH / extraction_strategy / aoi / f'orbit_{orbit}'
    fps += list(folder.glob('*.nc'))
    break

with h5py.File(f'data_{extraction_strategy}.h5', 'w') as f:
    groups = {f'fold{i}': f.create_group(f'fold{i}') for i in range(1,6)}
    groups['test'] = f.create_group('test')

    for fp in tqdm(fps):
        ts_full = xr.open_dataarray(fp)
        group_key = 'test' if ts.aoi in AOIS_TEST else f'fold{ts.bin}'
        grp = groups[group_key]

        ts_pos = ts_full.sel(date=slice('2021-02-24', '2023-02-23')) # two years, spanning the war
        ts_neg = ts_full.sel(date=slice('2020-02-24', '2022-02-23')) # two years, before the war
        for ts, label in [(ts_pos, 1), (ts_neg, 0)]:
            ds_name = f"{ts.aoi}_orbit_{ts.orbit}_{ts.unosat_id}_{label}"
            ds = grp.create_dataset(ds_name, data=ts.values)

            if label:
                id_left_bound = np.max(np.where(ts.date < np.datetime64(UKRAINE_WAR_START))) # closest date before the war
                id_right_bound = np.min(np.where(ts.date > np.datetime64(ts.date_of_analysis))) # closest date after the date of analysis
            else:
                id_left_bound = -1
                id_right_bound = -1

            ds.attrs['label'] = label
            ds.attrs['id_left_bound'] = id_left_bound
            ds.attrs['id_right_bound'] = id_right_bound
            for attr in ts.attrs:
                ds.attrs[attr] = ts.attrs[attr]


In [ ]:
with h5py.File(f'data_{extraction_strategy}.h5', 'r') as f:
    ts_pos = f['fold1']['UKR1_orbit_43_11131_1'][...]
    ts_neg = f['fold1']['UKR1_orbit_43_11131_0'][...]

In [ ]:
ts = read_ts('UKR1', 43, 11131)
ts

In [ ]:
with h5py.File(f'test.h5', 'w') as f:
    grp = f.create_group('test')
    ds = grp.create_dataset('test', data=ts.values)
    ds.attrs['label'] = np.inf

In [ ]:
item

In [ ]:
from src.data.utils import get_random_ts
from src.visualization.time_series import plot_ts
import matplotlib.pyplot as plt
from src.constants import AOIS_TEST
fig, ax = plt.subplots(5,2, figsize=(20,10))
for i in range(10):
    ts = get_random_ts(aoi=AOIS_TEST, seed=i+1035)
    plot_ts(ts, ax=ax[i//2, i%2])
plt.show()

In [ ]:
AOIS_TEST

In [ ]:
good_ts = [
    ('UKR15', 65, 15487),
    ('UKR15', 116, 15467),
    ('UKR15', 65, 15503),
    ('UKR15', 65, 15477),
    ('UKR6', 14, 14220),
    ('UKR6', 87, 14807),
    ('UKR8', 36, 13829),
    ('UKR6',36,14087)
]

In [ ]:
from src.data.utils import read_ts
from src.constants import DATA_PATH
extraction_strategy = 'pixel-wise-3x3'
base_folder = DATA_PATH / 'tmp' / f'cherrys_{extraction_strategy}'
folder = base_folder / 'easy_ts_xarray'
folder.mkdir(exist_ok=True, parents=True)
for i, (aoi, orbit, unosat_id) in enumerate(good_ts):
    ts = read_ts(aoi, orbit, unosat_id, extraction_strategy)
    ts = ts.sel(date=slice('2021-02-24', '2023-02-23'))
    ts.to_netcdf(folder / f'{aoi}_orbit_{orbit}_{unosat_id}.nc')

folder = base_folder / 'easy_ts_xarray_shifted'
folder.mkdir(exist_ok=True)
for i, (aoi, orbit, unosat_id) in enumerate(good_ts):
    ts = read_ts(aoi, orbit, unosat_id, extraction_strategy)
    ts = ts.sel(date=slice('2020-11-24', '2022-11-23'))
    ts.to_netcdf(folder / f'{aoi}_orbit_{orbit}_{unosat_id}.nc')

folder = base_folder / 'easy_ts_xarray_neg'
folder.mkdir(exist_ok=True)
for i, (aoi, orbit, unosat_id) in enumerate(good_ts):
    ts = read_ts(aoi, orbit, unosat_id, extraction_strategy)
    ts = ts.sel(date=slice('2020-02-24', '2022-02-23'))
    ts.to_netcdf(folder / f'{aoi}_orbit_{orbit}_{unosat_id}.nc')

In [ ]:
from src.data.utils import read_ts
from pathlib import Path
folder = Path('./easy_ts_before_war')
folder.mkdir(exist_ok=True)
fig, ax = plt.subplots(4,2, figsize=(20,10))
for i, (aoi, orbit, unosat_id) in enumerate(good_ts):
    ts = read_ts(aoi, orbit, unosat_id)
    ts = ts.sel(date=slice('2020-02-24', '2022-02-23'))
    np.save(folder / f'{aoi}_orbit_{orbit}_{unosat_id}.npy', ts.values)
    plot_ts(ts, ax=ax[i//2, i%2])
plt.tight_layout()

In [ ]:
# save as npy
import numpy as np


In [ ]:
plt.plot(np.load('ts.npy'))

In [ ]:
from src.constants import DATA_PATH
cherry_folder = DATA_PATH / "tmp" / "cherrys" / "easy_ts_xarray"
cherrys_fp = list(cherry_folder.glob("*.nc"))

from src.main_torben import Network
model = Network().cuda()

from src.data.utils import read_ts_from_fp
cherrys = [read_ts_from_fp(fp) for fp in cherrys_fp]

In [ ]:
cherry = cherrys[0]
cherry

In [ ]:
import torch
cherry_ = torch.tensor(cherry.to_numpy()).cuda().unsqueeze(0).float()

In [ ]:
pred = model(cherry_)
pred

In [ ]:
cherry[0,0]

In [ ]:
from src.visualization.time_series import plot_ts

ax = plot_ts(cherry)
ax2 = ax.twinx()
ax2.plot(cherry.date, pred.detach().cpu().numpy().squeeze(), color='r')